## install

In [1]:
!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install .
%cd /content/
!git clone https://huggingface.co/vinai/phobert-base

Cloning into 'transformers'...
remote: Enumerating objects: 134871, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 134871 (delta 28), reused 41 (delta 13), pack-reused 134791
Receiving objects: 100% (134871/134871), 132.11 MiB | 20.46 MiB/s, done.
Resolving deltas: 100% (101614/101614), done.
Updating files: 100% (3167/3167), done.
/content/transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 73.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.28.0.dev0-py3-none-any.whl size=6858427 sha256=3ce9bd95c757a3ea32fa3d27fc869e32fedba1bf0fcc

In [21]:
import torch
from transformers import AutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
import random
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
import numpy as np

bert = AutoModel.from_pretrained("/content/phobert-base")
bert_db = AutoModel.from_pretrained("/content/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("/content/phobert-base")

# print(vars(bert))

Some weights of the model checkpoint at /content/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at /content/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.de

In [20]:
sentence = 'Tôi chưa hiểu tại sao nó ra có 1 ký          tự'
tok = tokenizer.encode(sentence)

input_ids = torch.tensor(np.array([tok, tok, tok, tok])).to('cpu')
print(input_ids.shape)

with torch.no_grad():
    features = bert(input_ids, 
            token_type_ids=None)

print(features[0].shape)

torch.Size([4, 13])
torch.Size([4, 13, 768])


## from scratch

In [ ]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import tqdm


class Counter:
    def __init__(self):
        f = open("/content/phobert-base/vocab.txt", "r")
        vocab = f.read()
        vocab = vocab.split("\n")
        self.word2id = {}
        for v in vocab:
            v = v.split(" ")
            self.word2id[v[0]] = v[1]
 
    def __getitem__(self):
        return self.word2id

class IMDBBertDataset(Dataset):
    # Define Special tokens as attributes of class
    CLS = '[CLS]'
    PAD = '[PAD]'
    SEP = '[SEP]'
    MASK = '[MASK]'
    UNK = '[UNK]'

    MASK_PERCENTAGE = 0.15  # How much words to mask

    MASKED_INDICES_COLUMN = 'masked_indices'
    TARGET_COLUMN = 'indices'
    NSP_TARGET_COLUMN = 'is_next'
    TOKEN_MASK_COLUMN = 'token_mask'

    OPTIMAL_LENGTH_PERCENTILE = 70

    def __init__(self, path, tokenizer, ds_from=None, ds_to=None):
        self.ds: pd.Series = pd.read_csv(path)['review']

        if ds_from is not None or ds_to is not None:
            self.ds = self.ds[ds_from:ds_to]

        self.tokenizer = tokenizer
        self.counter = Counter()
        self.vocab = None

        self.optimal_sentence_length = None

        self.columns = [self.MASKED_INDICES_COLUMN, self.TARGET_COLUMN, self.TOKEN_MASK_COLUMN,
                        self.NSP_TARGET_COLUMN]
        self.df = self.prepare_dataset()

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        ...
    
    def prepare_dataset(self):
        sentences = []  
        nsp = []  
        sentence_lens = []

        # Split dataset on sentences
        for review in self.ds:
            review_sentences = review.split('. ')
            sentences += review_sentences
            self._update_length(review_sentences, sentence_lens)
        self.optimal_sentence_length = self._find_optimal_sentence_length(sentence_lens)

        for sentence in tqdm(sentences):  
            s = self.tokenizer(sentence)  
            self.counter.update(s)  
        
        self._fill_vocab()

    def _find_optimal_sentence_length(self, lengths):  
        arr = np.array(lengths)  
        return int(np.percentile(arr, self.OPTIMAL_LENGTH_PERCENTILE))

    def _fill_vocab(self):  
    # specials= argument is only in 0.12.0 version  
    # specials=[self.CLS, self.PAD, self.MASK, self.SEP, self.UNK]
        self.vocab = vocab(self.counter, min_freq=2)  

        # 0.11.0 uses this approach to insert specials  
        self.vocab.insert_token(self.CLS, 0)  
        self.vocab.insert_token(self.PAD, 1)  
        self.vocab.insert_token(self.MASK, 2)  
        self.vocab.insert_token(self.SEP, 3)  
        self.vocab.insert_token(self.UNK, 4)  
        self.vocab.set_default_index(4)

    def _create_item(self, first, second, target):  
    # Create masked sentence item  
        updated_first, first_mask = self._preprocess_sentence(first.copy())  
        updated_second, second_mask = self._preprocess_sentence(second.copy())
        nsp_sentence = updated_first + [self.SEP] + updated_second  
        nsp_indices = self.vocab.lookup_indices(nsp_sentence)  
        inverse_token_mask = first_mask + [True] + second_mask

##Read data


In [9]:
data_path = '/content/news_5000_19032020_201604.csv'

df = pd.read_csv(data_path)
print(df.columns)

df['title'] = df['title'].str.replace('\W', ' ').replace('  ', ' ')
df['description'] = df['description'].str.replace('\W', ' ').str.replace('  ', ' ')
df['content'] = df['content'].str.replace('\W', ' ').str.replace('  ', ' ')
df["id"] = df.index

Index(['url', 'title', 'keywords', 'description', 'content', 'len', 'author'], dtype='object')


<ipython-input-9-d82cc5bfae06>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title'] = df['title'].str.replace('\W', ' ').replace('  ', ' ')
<ipython-input-9-d82cc5bfae06>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['description'] = df['description'].str.replace('\W', ' ').str.replace('  ', ' ')
<ipython-input-9-d82cc5bfae06>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('\W', ' ').str.replace('  ', ' ')


In [ ]:
print(df)

In [ ]:
print(sum([len(i) for i in data_content])/len(data_content))

1773.3508


In [10]:
des_data = df[['description', 'id']].to_numpy()
tit_data = df[['title', 'id']].to_numpy()
queues = np.concatenate((des_data, tit_data), axis=0)

database = df['content'].to_numpy()

In [11]:
train_data, val_data, train_label, val_label = train_test_split(queues[:,0], queues[:,1], test_size=0.8)

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_LEN = 120

def get_split(text1, length, margin):
    body_length = length - margin
    l_total = []
    l_parcial = []
    if len(text1.split())//body_length > 0:
        n = len(text1.split())//body_length
    else: 
        n = 1

    for w in range(n):
        if w == 0:
            l_parcial = text1.split()[:length]
            l_total.append(" ".join(l_parcial))
        else:
            l_parcial = text1.split()[w * body_length:w * body_length + length]
            l_total.append(" ".join(l_parcial))
    return l_total

def encode(x):
    sents = []
    max_len = 0
    for sent in x:
        token = []
        for con_split in get_split(sent, 100, 30):
            token.append(tokenizer.encode(con_split))
            l = len(tokenizer.encode(con_split))
            if l > MAX_LEN:
                max_len += 1
        sents.append(token)

    print(max_len/len(x))
    return sents

train_ids = [tokenizer.encode(t) for t in train_data]
val_ids = [tokenizer.encode(t) for t in val_data]
database_ids = encode(database)
    
train_ids = pad_sequences(train_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
val_ids = pad_sequences(val_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
database_ids = [pad_sequences(d, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post") for d in database_ids]

0.0274


In [ ]:
print(database_ids[0])

In [13]:
def create_mask(x, is_content):
    result = []
    for sent in x:
        if is_content:
            masks = []
            for s in sent:
                mask = [int(token_id > 0) for token_id in s]
                masks.append(mask)
            result.append(masks)
        else:
            mask = [int(token_id > 0) for token_id in sent]
            result.append(mask)
    return result


train_masks = create_mask(train_ids, False)
val_masks = create_mask(val_ids, False)
database_masks = create_mask(database_ids, True)

In [ ]:
print(type(train_label[0]))

<class 'int'>


In [14]:
train_inputs = torch.tensor(train_ids)
val_inputs = torch.tensor(val_ids)
train_labels = torch.tensor(np.array([t for t in train_label]))
val_labels = torch.tensor(np.array([t for t in val_label]))
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=32)

In [ ]:
for step, batch in enumerate(train_dataloader):
    print(step)
    print(batch)

0
[tensor([[    0, 11016,  2680,  ...,     0,     0,     0],
        [    0,  6649,   376,  ...,     0,     0,     0],
        [    0,   125,  2406,  ...,     0,     0,     0],
        ...,
        [    0,   519,  7303,  ...,     0,     0,     0],
        [    0,   345,   258,  ...,     0,     0,     0],
        [    0, 24329,  7377,  ...,     0,     0,     0]]), tensor([[0, 1, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        ...,
        [0, 1, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0]]), tensor([1398,  481, 4653,  774, 4750, 1711, 3980, 4559, 3453, 3641, 4396,  582,
         635, 3717, 2511, 1872, 3363, 4939,  614, 2333, 1305,  623, 3004,  447,
        3221,  282, 1478, 4194, 2218,  871, 3014, 1067])]
1
[tensor([[    0,  3378,   163,  ...,     0,     0,     0],
        [    0,   358,   101,  ...,     0,     0,     0],
        [    0,  9284,   961,  ...,     0,     0,     0],
        ...,
       

## Training

In [15]:
from tqdm import tqdm_notebook
from torch.optim.adamw import AdamW

In [ ]:
bert.cuda()

In [16]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [17]:
import sys

device = 'cpu'
epochs = 10

param_optimizer = list(bert.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)


for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    total_loss = 0
    bert.train()
    train_accuracy = 0
    nb_train_steps = 0
    train_f1 = 0
    
    for step, batch in tqdm_notebook(enumerate(train_dataloader)):
        print(batch[0].shape)
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        bert.zero_grad()
        outputs = bert(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask)
        loss = outputs[0]
        print(outputs[0].shape)
        sys.exit()
        total_loss += loss.item()
        
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_train_accuracy, tmp_train_f1 = flat_accuracy(logits, label_ids)
        train_accuracy += tmp_train_accuracy
        train_f1 += tmp_train_f1
        nb_train_steps += 1
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(bert.parameters(), 1.0)
        optimizer.step()
        
    avg_train_loss = total_loss / len(train_dataloader)
    print(" Accuracy: {0:.4f}".format(train_accuracy/nb_train_steps))
    print(" F1 score: {0:.4f}".format(train_f1/nb_train_steps))
    print(" Average training loss: {0:.4f}".format(avg_train_loss))

    print("Running Validation...")
    bert.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    eval_f1 = 0
    for batch in tqdm_notebook(val_dataloader):

        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = bert(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask)
            logits = outputs[0]
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            tmp_eval_accuracy, tmp_eval_f1 = flat_accuracy(logits, label_ids)

            eval_accuracy += tmp_eval_accuracy
            eval_f1 += tmp_eval_f1
            nb_eval_steps += 1
    print(" Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
    print(" F1 score: {0:.4f}".format(eval_f1/nb_eval_steps))
print("Training complete!")


======== Epoch 1 / 10 ========
Training...


<ipython-input-17-dbea5ba9d376>:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step, batch in tqdm_notebook(enumerate(train_dataloader)):


0it [00:00, ?it/s]

torch.Size([32, 120])
torch.Size([32, 120, 768])


SystemExit: ignored

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
print(bert)

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop